In [1]:
import pandas as pd
import numpy as np

In [2]:
combined_data = pd.read_csv('data/prioritization_data/combined_data.csv')

In [3]:
combined_data.shape

(10836, 33)

In [4]:
combined_data.columns

Index(['Unnamed: 0', 'parentname_x', 'acctcode', 'attrition_prob_x',
       'last_52_weeks_gallons_x', 'monthly_avg_gallons_last_52_weeks_x',
       'max_gallons_per_month_x', 'last_52_weeks_gallons_max_x',
       'last_52_weeks_revenue_x', 'monthly_avg_revenue_last_52_weeks_x',
       'max_revenue_per_month_x', 'last_52_weeks_revenue_max_x',
       'parentname_y', 'attrition_prob_y', 'last_52_weeks_gallons_y',
       'monthly_avg_gallons_last_52_weeks_y', 'max_gallons_per_month_y',
       'last_52_weeks_gallons_max_y', 'last_4_weeks_gallons',
       'last_8_weeks_gallons', 'last_13_weeks_gallons',
       'last_26_weeks_gallons', 'last_52_weeks_revenue_y',
       'monthly_avg_revenue_last_52_weeks_y', 'max_revenue_per_month_y',
       'last_52_weeks_revenue_max_y', 'last_transaction_week',
       'last_transaction_month', 'count_sf_cases_last3m', 'phone',
       'mobilephone', 'otherphone', 'Current_account_managed'],
      dtype='object')

In [5]:
parent_level_df = combined_data[['parentname_x', 'acctcode', 'attrition_prob_x',
       'last_52_weeks_gallons_x', 'monthly_avg_gallons_last_52_weeks_x',
       'max_gallons_per_month_x', 'last_52_weeks_gallons_max_x',
       'last_52_weeks_revenue_x', 'monthly_avg_revenue_last_52_weeks_x',
       'max_revenue_per_month_x', 'last_52_weeks_revenue_max_x']]

In [6]:
#remove duplicate parentnames
parent_level_df = parent_level_df[~parent_level_df['parentname_x'].duplicated()]

In [7]:
#remove null values
parent_level_df = parent_level_df[~parent_level_df['last_52_weeks_gallons_x'].isna()]

In [8]:
#remove 0 week gallons
parent_level_df = parent_level_df[parent_level_df['last_52_weeks_gallons_x'] != 0]

In [9]:
parent_level_df.reset_index(drop = True, inplace = True)

In [10]:
parent_level_df.columns

Index(['parentname_x', 'acctcode', 'attrition_prob_x',
       'last_52_weeks_gallons_x', 'monthly_avg_gallons_last_52_weeks_x',
       'max_gallons_per_month_x', 'last_52_weeks_gallons_max_x',
       'last_52_weeks_revenue_x', 'monthly_avg_revenue_last_52_weeks_x',
       'max_revenue_per_month_x', 'last_52_weeks_revenue_max_x'],
      dtype='object')

In [11]:
parent_level_df[parent_level_df['parentname_x'] == 'SUTTON TRUCKING']

,parentname_x,acctcode,attrition_prob_x,last_52_weeks_gallons_x,monthly_avg_gallons_last_52_weeks_x,max_gallons_per_month_x,last_52_weeks_gallons_max_x,last_52_weeks_revenue_x,monthly_avg_revenue_last_52_weeks_x,max_revenue_per_month_x,last_52_weeks_revenue_max_x
4078,SUTTON TRUCKING,L-17A,0.9132,2195.72,182.9766,307.1,3685.2,266.97,22.2475,52.4,628.8


#### Calculate Percentile ranks for parent_level_data

In [12]:
parent_level_df['last_52_weeks_gallons_rank%'] = parent_level_df['last_52_weeks_gallons_x'].rank(pct = True)

In [13]:
parent_level_df['last_52_weeks_gallons_max_rank%'] = parent_level_df['last_52_weeks_gallons_max_x'].rank(pct = True)

In [14]:
parent_level_df['last_52_weeks_revenue_rank%'] = parent_level_df['last_52_weeks_revenue_x'].rank(pct = True)

In [15]:
parent_level_df['last_52_weeks_revenue_max_rank%'] = parent_level_df['last_52_weeks_revenue_max_x'].rank(pct = True)

In [16]:
#Max of all ranks
parent_level_df['max_rank%'] = parent_level_df[['last_52_weeks_gallons_rank%', 'last_52_weeks_gallons_max_rank%', 'last_52_weeks_revenue_rank%','last_52_weeks_revenue_max_rank%']].max(axis = 1)

In [17]:
#Create Priority 1
parent_level_df['Priority_1'] = parent_level_df['attrition_prob_x'] * parent_level_df['max_rank%']

In [18]:
# Define bins and labels to categorize 'attrition_prob' & 'priority_1'
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
labels = ['0-0.2', '0.2-0.4', '0.4-0.6', '0.6-0.8', '0.8-1']

In [19]:
parent_level_df.columns

Index(['parentname_x', 'acctcode', 'attrition_prob_x',
       'last_52_weeks_gallons_x', 'monthly_avg_gallons_last_52_weeks_x',
       'max_gallons_per_month_x', 'last_52_weeks_gallons_max_x',
       'last_52_weeks_revenue_x', 'monthly_avg_revenue_last_52_weeks_x',
       'max_revenue_per_month_x', 'last_52_weeks_revenue_max_x',
       'last_52_weeks_gallons_rank%', 'last_52_weeks_gallons_max_rank%',
       'last_52_weeks_revenue_rank%', 'last_52_weeks_revenue_max_rank%',
       'max_rank%', 'Priority_1'],
      dtype='object')

In [20]:
# Create a new column with the attrition probability & priority categories
parent_level_df['attrition_prob_bin'] = pd.cut(parent_level_df['attrition_prob_x'], bins=bins, labels=labels, include_lowest=True)
parent_level_df['Priority_1_bin'] = pd.cut(parent_level_df['Priority_1'], bins=bins, labels=labels, include_lowest=True)
parent_level_df['revenue_bin'] = pd.cut(parent_level_df['last_52_weeks_revenue_rank%'], bins=bins, labels=labels, include_lowest=True)

In [21]:
parent_level_df['revenue_bin']

0       0.4-0.6
1       0.4-0.6
2       0.6-0.8
3       0.6-0.8
4         0-0.2
         ...   
7488    0.4-0.6
7489      0-0.2
7490    0.4-0.6
7491    0.4-0.6
7492    0.6-0.8
Name: revenue_bin, Length: 7493, dtype: category
Categories (5, object): ['0-0.2' < '0.2-0.4' < '0.4-0.6' < '0.6-0.8' < '0.8-1']

In [22]:
# Group by 'priority_1' and 'attrition_prob_bin' and count the occurrences of 'parent_name'
priority_1_grouped = parent_level_df.groupby(['Priority_1_bin', 'attrition_prob_bin']).agg(count_of_parent_name=('parentname_x', 'count')).reset_index()
# Groupby revenue 
attrition_revenue_grouped = parent_level_df.groupby(['revenue_bin', 'attrition_prob_bin']).agg(sum_of_revenue=('last_52_weeks_revenue_x', 'sum')).reset_index()


#### Create pivot tables

In [23]:
#create pivot table for priority_1 
pivot_priority_1 = pd.pivot_table(priority_1_grouped, index='Priority_1_bin', columns='attrition_prob_bin', 
                             values='count_of_parent_name', aggfunc='sum', margins=True, margins_name='Grand Total')
pivot_revenue_dist = pd.pivot_table(attrition_revenue_grouped, index='revenue_bin', columns='attrition_prob_bin', 
                             values='sum_of_revenue', aggfunc='sum', margins=True, margins_name='Grand Total')

In [24]:
pivot_priority_1

attrition_prob_bin,0-0.2,0.2-0.4,0.4-0.6,0.6-0.8,0.8-1,Grand Total
Priority_1_bin,,,,,,
0-0.2,1494,682,344,457,440,3417
0.2-0.4,0,629,848,841,468,2786
0.4-0.6,0,0,241,494,293,1028
0.6-0.8,0,0,0,92,136,228
0.8-1,0,0,0,0,34,34
Grand Total,1494,1311,1433,1884,1371,7493


In [25]:
pivot_priority_1.columns

Index(['0-0.2', '0.2-0.4', '0.4-0.6', '0.6-0.8', '0.8-1', 'Grand Total'], dtype='object', name='attrition_prob_bin')

In [26]:
pivot_priority_1.reset_index(drop = False, inplace = True)

In [27]:
pivot_priority_1

attrition_prob_bin,Priority_1_bin,0-0.2,0.2-0.4,0.4-0.6,0.6-0.8,0.8-1,Grand Total
0,0-0.2,1494,682,344,457,440,3417
1,0.2-0.4,0,629,848,841,468,2786
2,0.4-0.6,0,0,241,494,293,1028
3,0.6-0.8,0,0,0,92,136,228
4,0.8-1,0,0,0,0,34,34
5,Grand Total,1494,1311,1433,1884,1371,7493


In [28]:
priority_column_names = {
    'Priority_1_bin':'priority_bin',
    '0-0.2':'attrition_bin_0_0_2',
    '0.2-0.4':'attrition_bin_0_2_0_4',
    '0.4-0.6':'attrition_bin_0_4_0_6',
    '0.6-0.8':'attrition_bin_0_6_0_8',
    '0.8-1':'attrition_bin_0_8_1',
    'Grand Total':'grand_total'
}

In [29]:
pivot_priority_1.rename(columns = priority_column_names,inplace = True) 

In [30]:
pivot_priority_1

attrition_prob_bin,priority_bin,attrition_bin_0_0_2,attrition_bin_0_2_0_4,attrition_bin_0_4_0_6,attrition_bin_0_6_0_8,attrition_bin_0_8_1,grand_total
0,0-0.2,1494,682,344,457,440,3417
1,0.2-0.4,0,629,848,841,468,2786
2,0.4-0.6,0,0,241,494,293,1028
3,0.6-0.8,0,0,0,92,136,228
4,0.8-1,0,0,0,0,34,34
5,Grand Total,1494,1311,1433,1884,1371,7493


In [31]:
#revenue distribution
pivot_revenue_dist

attrition_prob_bin,0-0.2,0.2-0.4,0.4-0.6,0.6-0.8,0.8-1,Grand Total
revenue_bin,,,,,,
0-0.2,-2457.00,15779.11,57536.34,130803.09,152543.80,354205.34
0.2-0.4,29183.60,123046.28,195237.59,339910.69,204311.90,891690.06
0.4-0.6,199514.70,392030.42,474185.57,466211.31,223619.00,1755561.00
0.6-0.8,1158270.87,1173783.97,827722.82,667919.56,248571.96,4076269.18
0.8-1,35222761.06,3588039.15,1680429.96,919911.71,724544.71,42135686.59
Grand Total,36607273.23,5292678.93,3235112.28,2524756.36,1553591.37,49213412.17


In [32]:
pivot_revenue_dist.reset_index(drop = False, inplace = True)

In [33]:
pivot_revenue_dist.columns

Index(['revenue_bin', '0-0.2', '0.2-0.4', '0.4-0.6', '0.6-0.8', '0.8-1',
       'Grand Total'],
      dtype='object', name='attrition_prob_bin')

In [34]:
revenue_column_names = {
    'revenue_bin':'revenue_bin',
    '0-0.2':'attrition_bin_0_0_2',
    '0.2-0.4':'attrition_bin_0_2_0_4',
    '0.4-0.6':'attrition_bin_0_4_0_6',
    '0.6-0.8':'attrition_bin_0_6_0_8',
    '0.8-1':'attrition_bin_0_8_1',
    'Grand Total':'grand_total'
}

In [35]:
pivot_revenue_dist.rename(columns = revenue_column_names,inplace = True) 

In [36]:
pivot_revenue_dist

attrition_prob_bin,revenue_bin,attrition_bin_0_0_2,attrition_bin_0_2_0_4,attrition_bin_0_4_0_6,attrition_bin_0_6_0_8,attrition_bin_0_8_1,grand_total
0,0-0.2,-2457.00,15779.11,57536.34,130803.09,152543.80,354205.34
1,0.2-0.4,29183.60,123046.28,195237.59,339910.69,204311.90,891690.06
2,0.4-0.6,199514.70,392030.42,474185.57,466211.31,223619.00,1755561.00
3,0.6-0.8,1158270.87,1173783.97,827722.82,667919.56,248571.96,4076269.18
4,0.8-1,35222761.06,3588039.15,1680429.96,919911.71,724544.71,42135686.59
5,Grand Total,36607273.23,5292678.93,3235112.28,2524756.36,1553591.37,49213412.17


In [37]:
pivot_revenue_dist.to_csv('data/prioritization_data/revenue_pivot.csv', index = False, index_label = False)

In [38]:
pivot_priority_1.to_csv('data/prioritization_data/priority_pivot.csv', index = False, index_label = False)

#### Rank Parents

In [39]:
def rank_parents(parent_level_df):
    labels = ['0.8-1', '0.6-0.8', '0.4-0.6', '0.2-0.4', '0-0.2']
    rank = 0
    for label_1 in labels:
        # 1st Filter rows where Priority_1_bin equals label_1
        filtered_rows_1 = parent_level_df[parent_level_df['Priority_1_bin'] == label_1]
        #2nd filter where attrition_prob_bin equals label_2
        for label_2 in labels:
            filtered_rows_2 = filtered_rows_1[filtered_rows_1['attrition_prob_bin'] == label_2]
            #Rank all non-empty parents
            if not filtered_rows_2.empty:
                rank +=1
                parent_names = filtered_rows_2['parentname_x'].values
                parent_level_df.loc[parent_level_df['parentname_x'].isin(parent_names), 'parent_rank'] = rank
    return parent_level_df

In [40]:
final_parent_level_df = rank_parents(parent_level_df)

In [41]:
final_parent_level_df

,parentname_x,acctcode,attrition_prob_x,last_52_weeks_gallons_x,monthly_avg_gallons_last_52_weeks_x,max_gallons_per_month_x,last_52_weeks_gallons_max_x,last_52_weeks_revenue_x,monthly_avg_revenue_last_52_weeks_x,max_revenue_per_month_x,...,last_52_weeks_gallons_rank%,last_52_weeks_gallons_max_rank%,last_52_weeks_revenue_rank%,last_52_weeks_revenue_max_rank%,max_rank%,Priority_1,attrition_prob_bin,Priority_1_bin,revenue_bin,parent_rank
0,"REINSFELDER, INC.",RE215,0.0092,21192.48,1766.0400,3700.36,44404.32,1359.78,113.3150,445.51,...,0.495930,0.535166,0.548512,0.623515,0.623515,0.005736,0-0.2,0-0.2,0.4-0.6,15.0
1,"SMOCK, INC.",SP990,0.8832,69080.48,5756.7066,23937.97,287255.64,1030.02,85.8350,149.99,...,0.720272,0.845456,0.470706,0.328240,0.845456,0.746707,0.8-1,0.6-0.8,0.4-0.6,2.0
2,U & R CARRIER INC,G-93N,0.5384,28508.53,2375.7108,3634.09,43609.08,1799.85,149.9875,282.28,...,0.559722,0.529561,0.621246,0.515147,0.621246,0.334479,0.4-0.6,0.2-0.4,0.6-0.8,9.0
3,COTNER TRANSPORT INC,CT278,0.2334,192990.61,16082.5508,21015.33,252183.96,3807.88,317.3233,1656.70,...,0.840518,0.830375,0.774323,0.847725,0.847725,0.197859,0.2-0.4,0-0.2,0.6-0.8,14.0
4,TREE LINE TRANSPORTATION INC,TL160,0.7728,2273.31,189.4425,695.63,8347.56,163.37,13.6141,25.00,...,0.059255,0.098492,0.048045,0.030695,0.098492,0.076115,0.6-0.8,0-0.2,0-0.2,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7488,"AUTOMOTIVE CORE SUPPLY, INC",AW825,0.6620,5934.13,494.5108,1576.58,18918.96,1372.17,114.3475,621.22,...,0.178567,0.289737,0.551448,0.691712,0.691712,0.457914,0.6-0.8,0.4-0.6,0.4-0.6,5.0
7489,FT TRANSPORTATION INC,W-01K,0.3053,10951.54,912.6283,1432.13,17185.56,264.00,22.0000,42.50,...,0.321500,0.259976,0.111704,0.075003,0.321500,0.098154,0.2-0.4,0-0.2,0-0.2,14.0
7490,VAL'S CARGO LLC,G-56Q,0.3531,58214.14,4851.1783,6953.79,83445.48,1304.54,108.7116,256.25,...,0.694915,0.672094,0.537301,0.492993,0.694915,0.245375,0.2-0.4,0.2-0.4,0.4-0.6,10.0
7491,UNIVERSAL GROUND LOGISTICS,J-144,0.7923,16986.77,1415.5641,2129.14,25549.68,835.82,69.6516,141.14,...,0.440411,0.384359,0.404778,0.309422,0.440411,0.348938,0.6-0.8,0.2-0.4,0.4-0.6,8.0


In [42]:
final_parent_level_df.columns

Index(['parentname_x', 'acctcode', 'attrition_prob_x',
       'last_52_weeks_gallons_x', 'monthly_avg_gallons_last_52_weeks_x',
       'max_gallons_per_month_x', 'last_52_weeks_gallons_max_x',
       'last_52_weeks_revenue_x', 'monthly_avg_revenue_last_52_weeks_x',
       'max_revenue_per_month_x', 'last_52_weeks_revenue_max_x',
       'last_52_weeks_gallons_rank%', 'last_52_weeks_gallons_max_rank%',
       'last_52_weeks_revenue_rank%', 'last_52_weeks_revenue_max_rank%',
       'max_rank%', 'Priority_1', 'attrition_prob_bin', 'Priority_1_bin',
       'revenue_bin', 'parent_rank'],
      dtype='object')

In [43]:
final_parent_level_df_copy = final_parent_level_df[['parentname_x','Priority_1', 'attrition_prob_bin', 'Priority_1_bin',
       'revenue_bin', 'parent_rank']]

#### Merge parent priority columns with original combined data

In [44]:
final_parent_priority_df = pd.merge(combined_data,final_parent_level_df_copy, on = 'parentname_x')

In [45]:
final_parent_priority_df.to_csv('data/prioritization_data/final_parent_priority.csv')